In [ ]:
import numpy as np
import pandas as pd
import os
import os
os.chdir(r"C:\Users\Dell\OneDrive\jbooks")
print(os.getcwd())


C:\Users\Dell\OneDrive\jbooks


In [ ]:
df = pd.read_csv('hacktrain.csv')

In [ ]:
df['class'].value_counts()

class
forest        6159
farm           841
impervious     669
grass          196
water          105
orchard         30
Name: count, dtype: int64

In [ ]:
df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
df.isnull().sum()

ID               0
class            0
20150720_N     560
20150602_N    1200
20150517_N     800
20150501_N     960
20150415_N     480
20150330_N    1120
20150314_N     720
20150226_N    1360
20150210_N     640
20150125_N    1040
20150109_N     880
20141117_N    1280
20141101_N     400
20141016_N    1440
20140930_N     800
20140813_N     560
20140626_N    1600
20140610_N     480
20140525_N     720
20140509_N     880
20140423_N    1760
20140407_N     640
20140322_N    1120
20140218_N    1440
20140202_N     560
20140117_N    1200
20140101_N     400
dtype: int64

In [ ]:
from sklearn.impute import KNNImputer

features = df.drop(columns=['ID', 'class']).columns
imputer = KNNImputer(n_neighbors=5)

imputed_parts = []

for class_val, group in df.groupby('class'):
    temp = group.copy()
    imputed_values = imputer.fit_transform(temp[features])
    temp[features] = imputed_values
    imputed_parts.append(temp)

df = pd.concat(imputed_parts).sort_index()

In [ ]:
df.isnull().sum()

ID            0
class         0
20150720_N    0
20150602_N    0
20150517_N    0
20150501_N    0
20150415_N    0
20150330_N    0
20150314_N    0
20150226_N    0
20150210_N    0
20150125_N    0
20150109_N    0
20141117_N    0
20141101_N    0
20141016_N    0
20140930_N    0
20140813_N    0
20140626_N    0
20140610_N    0
20140525_N    0
20140509_N    0
20140423_N    0
20140407_N    0
20140322_N    0
20140218_N    0
20140202_N    0
20140117_N    0
20140101_N    0
dtype: int64

In [ ]:
from scipy.stats import zscore
filtered_df = df.copy()
def zs(data, categories):
    for col in data.drop(columns=['ID', 'class']).columns:
        for category in categories:
            subset = data[data['class'] == category]
            col_zscores = zscore(subset[col].dropna())
            z_indexed = pd.Series(col_zscores, index=subset[col].dropna().index)
            outlier_index = z_indexed[abs(z_indexed) > 2].index
            data = data.drop(index=outlier_index)
    return data

In [ ]:
categories = ['impervious', 'forest', 'farm', 'grass', 'orchard',"water"]
filtered_df = zs(filtered_df, categories)

In [ ]:
filtered_df['class'].value_counts()

class
forest        2980
farm           256
impervious     174
grass           69
water           18
orchard         14
Name: count, dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

label_encoder = LabelEncoder()
filtered_df['class'] = label_encoder.fit_transform(filtered_df['class'])

X = filtered_df.drop(columns=['class', 'ID'])
y = filtered_df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y)

model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=10000
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))

In [ ]:
test_data = pd.read_csv("hacktest.csv")
test_data.shape

(2845, 29)

In [ ]:
ID = test_data['ID']
test_data.drop(columns = ['Unnamed: 0', 'ID'], inplace=True)

In [ ]:
y_test = model.predict(test_data)
y_test = label_encoder.inverse_transform(y_test)

In [ ]:
result = pd.DataFrame({'ID': ID, 'class': y_test})
result.to_csv("output.csv", index=False)